In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

import sys
sys.path.append("../src")
from data_loader import *

In [ ]:
event="0210"
size ="128"
#a = np.load("../data/real/images/run_"+event+"_label_True_size_"+size+".npy")
x_set = np.load("../data/latent/clf_latent/vgg_data_repr.npy")
y_set = np.load("../data/latent/clf_latent/targets.npy")

In [ ]:
_, x, y = load_real_event("128")

In [ ]:
print(x_set[0].shape)

In [ ]:
which = 1
data = x_set[which]
targets = y_set[which]
true_class = targets.argmax(1)

pca_repr = PCA(50).fit_transform(data)

In [ ]:
dim_red = TSNE(2, perplexity=15, init="pca")
data_to_plot = dim_red.fit_transform(pca_repr)
#print(dim_red.explained_variance_ratio_)


In [ ]:
fig, ax = plt.subplots()
classes = ["Proton", "Carbon", "Other"]
for c in np.unique(true_class):
    w = true_class == c
    ax.scatter(data_to_plot[w][:,0], data_to_plot[w][:,1], alpha=0.4, label=classes[c])
    
plt.legend()

In [ ]:
def logreg(input_shape, n_classes=2, lmd=0.2):
    model = keras.models.Sequential()
    model.add(
        keras.layers.Dense(
        n_classes,
        input_shape=input_shape,
        kernel_regularizer="l2",
        )
    )
    model.add(
        keras.layers.BatchNormalization()
    )
    model.add(
        keras.layers.Activation("sigmoid")
    )
    return model

In [ ]:
def vgg_model(input_dim):
    input_layer = Input(shape=input_dim)
    vgg = VGG16(include_top=False, input_tensor=input_layer)
    which_o = 3
    o = Flatten()(vgg.layers[which_o].output)
    return Model(inputs=input_layer, outputs=o)
    
def resnet_model(input_dim):
    input_layer = Input(shape=input_dim)
    res_net = ResNet50(include_top=False, input_tensor=input_layer)
    o = Flatten()(res_net.output)
    return Model(inputs=input_layer, outputs=o)

In [ ]:
model = vgg_model((128, 128, 3))
model_repr = model.predict(np.concatenate([x, x, x], -1))

In [ ]:
model_repr.shape

In [ ]:
pca = PCA(1500, svd_solver="randomized")
pca.fit(model_repr)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pca.explained_variance_ratio_)

In [ ]:
pca_repr = pca.transform(model_repr)
#pca_vgg_test = pca.transform(vgg_model.predict(original_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

xtr, xte, ytr, yte = train_test_split(pca_repr, y)
clf_model = RandomForestClassifier(max_features=0.1, class_weight="balanced")
clf_model.fit(xtr, ytr)
print(clf_model.score(xte, yte))

In [ ]:
important = clf_model.feature_importances_ > 1e-2
plt.hist(clf_model.feature_importances_[important])
most_import = np.argsort(-clf_model.feature_importances_)[:2]
print(most_import)
pca_2dim = pca_repr[:, most_import]

In [ ]:
y_m = y.argmax(1)
classes = np.unique(y_m)
for c in classes:
    which = y_m == c
    plt.scatter(pca_2dim[which,0], pca_2dim[which,1], label=classes[c])
    
plt.legend()

In [ ]:
from sklearn.cluster import MiniBatchKMeans

cluster_model = MiniBatchKMeans(
    n_clusters=3,
    batch_size=150,
    n_init=100,
    )
cluster_model.fit(pca_vgg_train)
train_pred = cluster_model.predict(pca_vgg_train)
test_pred = cluster_model.predict(pca_vgg_test)

In [ ]:
def latent_distance(x, y, weight_func=lambda x: x):
    """
    x and y should  be T by L matrices 
    this function measures  euclidian distance along T
    and reduces to a float along L 
    """
    
    sub = x - y 
    sub = weight_func(sub)
    
    tmp = np.power(sub, 2)
    tmp = np.sum(tmp, axis=1)
    tmp = np.sqrt(tmp,)
    
    return np.sum(tmp)

def euclidian(x, y):
    return(np.sqrt(np.sum(np.power(x-y, 2))))

n_events = pca_vgg_train.shape[0]
train_dist_matrix = np.zeros((n_events, n_events))

n_test = pca_vgg_test.shape[0]
test_dist_matrix = np.zeros((n_test, n_test))

T = np.expand_dims(np.arange(pca_vgg_train.shape[0],), -1)
linear_weight = lambda x: x/(1 + T )

for i in range(n_events):
    for j in range(n_events):
        #dist_matrix[i, j] = latent_distance(original_latent[:, i, :], original_latent[:, j, :], weight_func=linear_weight)
        train_dist_matrix[i, j] = euclidian(pca_vgg_train[i, :], pca_vgg_train[j, :])
        

for i in range(n_test):
    for j in range(n_test):
        #dist_matrix[i, j] = latent_distance(original_latent[:, i, :], original_latent[:, j, :], weight_func=linear_weight)
        test_dist_matrix[i, j] = euclidian(pca_vgg_test[i, :], pca_vgg_test[j, :])
        

In [ ]:
%matplotlib notebook

flat_dist = train_dist_matrix.flatten()
flat_dist.sort()
plt.plot(flat_dist, "ko", alpha=0.4)


In [ ]:
from sklearn.cluster import DBSCAN

cluster_model = DBSCAN(
                eps=200,
                metric="precomputed",
                min_samples=8
            )

train_pred = cluster_model.fit_predict(train_dist_matrix)
#test_pred = cluster_model.transform(test_dist_matrix)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, confusion_matrix


a = plot_confusion_matrix(train_targets, train_pred, ["proton", "carbon", "junk"])
a.set_title("Confusion matrix for Train")

print("Scores on Train: ")
print("ARI : ", adjusted_rand_score(train_targets, train_pred))
print("NMI : ", normalized_mutual_info_score(train_targets, train_pred))


In [ ]:
a = plot_confusion_matrix(test_targets, test_pred, ["proton", "carbon", "junk"])
a.set_title("Confusion matrix for test")


print("Scores on Train: ")
print("ARI : ", adjusted_rand_score(test_targets, test_pred))
print("NMI : ", normalized_mutual_info_score(test_targets, test_pred))


In [ ]:
from sklearn.manifold import TSNE

projection = TSNE(2, perplexity=34, learning_rate=10).fit_transform(pca_vgg_train)


In [ ]:
proton = projection[train_targets==0]
carbon = projection[train_targets==1]
junk = projection[train_targets==2]

fig, ax = plt.subplots()
ax.scatter(proton[:,0], proton[:,1], c="r", alpha=0.6)
ax.scatter(carbon[:,0], carbon[:,1], c="g", alpha=0.2)
ax.scatter(junk[:,0], junk[:,1], c="b", alpha=0.2)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(projection[:,0], projection[:,1], alpha=0.2)
plt.show()